# Schema-based information retrieval from judgements


In [ ]:
# | eval: false
%load_ext autoreload
%autoreload 2

In [ ]:
# | eval: false
from loguru import logger

import mlflow

from juddges.prompts.information_extraction import prepare_information_extraction_chain, SCHEMA
from juddges.settings import prepare_langchain_cache, prepare_mlflow
from juddges.data.pl_court_api import PolishCourtAPI

In [ ]:
# | eval: false
from dotenv import load_dotenv

load_dotenv("../../.env")

True

In [ ]:
# |eval: false
prepare_langchain_cache()
prepare_mlflow()

In [ ]:
# | eval: false
logger.add(level="INFO", sink="../../logs/ir_judgments.log")

1

In [ ]:
LLM_NAME = "gpt-4-0125-preview"
PROMPT_VERSION = "initial-ir"
LANGUAGE = "Polish"

In [ ]:
# | eval: false
mlflow.start_run(run_name=f"{PROMPT_VERSION}-{LLM_NAME}")

<ActiveRun: >

In [ ]:
# | eval: false
api = PolishCourtAPI()

In [ ]:
# | eval: false
judgement = api.get_content(id="155000000001006_II_AKa_000334_2019_Uz_2020-02-06_001")

In [ ]:
# | eval: false
judgement

'<?xml version=\'1.0\' encoding=\'UTF-8\'?>\n<xPart xPublisherFullName="Anna Turek" xPublisher="anna.turek" xEditorFullName="Wiktoria Dąbrowicz" xEditor="wiktoria.dabrowicz" xVersion="1.0" xLang="PL" xFromPg="1" xToPage="2" xFlag="published" xVolType="15/500000/0001006/AKa" xYear="2019" xVolNmbr="000334" xDocType="Uz" xml:space="preserve">\n  <xName>Wyrok</xName>\n  <xBlock>\n    <xText xALIGNx="left">\n      <xBx>Sygn. akt II AKa 334/19</xBx></xText>\n    <xText/>\n    <xUnit xBold="true" xIsTitle="true" xType="part">\n      <xName>WYROK</xName>\n      <xTitle>W IMIENIU RZECZYPOSPOLITEJ POLSKIEJ</xTitle>\n      <xText xALIGNx="right">Dnia 6 lutego 2020 r.</xText>\n      <xText><xBx>Sąd Apelacyjny we Wrocławiu II Wydział Karny w składzie:</xBx></xText>\n      <xText>Przewodniczący: sędzia SA Cezariusz Baćkowski</xText>\n      <xText>Sędziowie: SA Piotr Kaczmarek (spr.)</xText>\n      <xText>SA Robert Zdych</xText>\n      <xText>Protokolant: Wiktoria Dąbrowicz</xText>\n      <xText><xBx

In [ ]:
# | eval: false
print(SCHEMA)


defendant_gender: string
defendant_age: integer
defendant_relationship_status: string
defendant_has_children: boolean
defendant_homeless: boolean
appellant: string
appeal_against: string
defendant_plead_or_convicted: string "guilty plea" or "convicted at trial"
jury_unanimous: string "unanimous" or "other"
first_trial: boolean
drug_offence: boolean
original_sentence: string
tried_court_type: string "Crown" or "magistrates'"
single_transaction_multiple_offence: boolean
multiple_transactions_period: string "within 1 year" or "more than 1 year"
concurrent_or_consecutive_sentence: string "concurrently" or "consecutively"
sentence_on_top_existing: boolean
sentence_adding_up: boolean
sentence_leniency: string "unduly lenient" or "too excessive"
guilty_plea_reduction_reason: string
sentence_discount_mention: boolean
totality_issues_similar_offences: boolean
sentence_proportionality_mention: boolean
sentence_type_issues_totality: boolean
sentence_adjustment_mention: boolean
offender_culpabili

In [ ]:
# | eval: false
chain = prepare_information_extraction_chain()

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
# | eval: false
logged_model = mlflow.langchain.log_model(chain, "langchain_model")

2024/03/27 12:12:33 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmppkf7491l/model, flavor: langchain). Fall back to return ['langchain==0.1.13', 'pydantic==2.6.4', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 


In [ ]:
# | eval: false
mlflow.log_param("prompt_version", PROMPT_VERSION)
mlflow.log_param("llm_name", LLM_NAME)
mlflow.log_param("language", LANGUAGE)
mlflow.log_param("schema", SCHEMA)

'\ndefendant_gender: string\ndefendant_age: integer\ndefendant_relationship_status: string\ndefendant_has_children: boolean\ndefendant_homeless: boolean\nappellant: string\nappeal_against: string\ndefendant_plead_or_convicted: string "guilty plea" or "convicted at trial"\njury_unanimous: string "unanimous" or "other"\nfirst_trial: boolean\ndrug_offence: boolean\noriginal_sentence: string\ntried_court_type: string "Crown" or "magistrates\'"\nsingle_transaction_multiple_offence: boolean\nmultiple_transactions_period: string "within 1 year" or "more than 1 year"\nconcurrent_or_consecutive_sentence: string "concurrently" or "consecutively"\nsentence_on_top_existing: boolean\nsentence_adding_up: boolean\nsentence_leniency: string "unduly lenient" or "too excessive"\nguilty_plea_reduction_reason: string\nsentence_discount_mention: boolean\ntotality_issues_similar_offences: boolean\nsentence_proportionality_mention: boolean\nsentence_type_issues_totality: boolean\nsentence_adjustment_mention:

In [ ]:
# | eval: false
retrieved_informations = chain.invoke({"LANGUAGE": LANGUAGE, "TEXT": judgement, "SCHEMA": SCHEMA})

In [ ]:
# | eval: false
retrieved_informations

{'defendant_gender': '',
 'defendant_age': 0,
 'defendant_relationship_status': '',
 'defendant_has_children': False,
 'defendant_homeless': False,
 'appellant': 'oskarżeni',
 'appeal_against': 'wyrok Sądu Okręgowego w Świdnicy z dnia 5 kwietnia 2019 r., sygn. akt III K 52/17',
 'defendant_plead_or_convicted': '',
 'jury_unanimous': '',
 'first_trial': False,
 'drug_offence': True,
 'original_sentence': '',
 'tried_court_type': '',
 'single_transaction_multiple_offence': False,
 'multiple_transactions_period': '',
 'concurrent_or_consecutive_sentence': '',
 'sentence_on_top_existing': False,
 'sentence_adding_up': False,
 'sentence_leniency': '',
 'guilty_plea_reduction_reason': '',
 'sentence_discount_mention': False,
 'totality_issues_similar_offences': False,
 'sentence_proportionality_mention': False,
 'sentence_type_issues_totality': False,
 'sentence_adjustment_mention': False,
 'offender_culpability_determination': False,
 'harm_caused_determination': False,
 'offence_seriousnes

In [ ]:
# | eval: false
from juddges.data.models import get_mongo_collection

judgements_collection = get_mongo_collection("judgements")

In [ ]:
# | eval: false
judgements = list(judgements_collection.find(filter={}).limit(100))

In [ ]:
# | eval: false
len(judgements)

100

In [ ]:
# | eval: false
print(judgements[0]["text"])

Sygn. akt : II AKa 211/99

WYROK
W IMIENIU RZECZYPOSPOLITEJ POLSKIEJ
Dnia 24 sierpnia 1999 r.
Sąd Apelacyjny w Katowicach II Wydział Karny w składzie:

Przewodniczący

SSA Stanisław Raszka

Sędziowie

SSA Wiesława Gawrońska (spr.)
SSA Jan Dybek

Protokolant:

Barbara Gawor

przy udziale Prokuratora Prok. Apel. Janusza Konstantego
po rozpoznaniu w dniu 24 sierpnia 1999 roku
sprawy z wnioskuL. O.o odszkodowanie i zadośćuczynienie dochodzone w trybie art. 8 ustęp 1 ustawy z dnia 23.02.1991r. (Dz. U. nr 34 poz. 149)
z powodu apelacji, wniesionej przez pełnomocnika wnioskodawczyni
od wyroku Sądu Okręgowego w Katowicach
z dnia 10 czerwca 1999r. sygn. akt XVI1 Ko 450/96

uchyla zaskarżony wyrok i przekazuje sprawę Sądowi Okręgowemu w Katowicach do ponownego rozpoznania.

II AKa 211/99

UZASADNIENIE
Sąd Okręgowy w Katowicach, wyrokiem z dnia 10 czerwca 1999 r. sygn.XVI 1 Ko 450/96 oddalił wniosekL. O.o zasądzenie odszkodowaniai zadośćuczynienia w oparciu o art. 8 ust. 1 ustawy z dnia 23 lutego

In [ ]:
# | eval: false
judgements_retrieved_informations = chain.batch(
    [
        {"LANGUAGE": LANGUAGE, "TEXT": judgement["text"], "SCHEMA": SCHEMA}
        for judgement in judgements[:10]
    ]
)